## Example 1: US Congress Dataset

The U.S. Congress dataset contains public information on historical members of Congress and illustrates several fundamental capabilities of .groupby().

https://realpython.com/pandas-groupby/

In [2]:
import pandas as pd

In [3]:
# define dtypes
dtypes = {
    "first_name": "category",
    "gender": "category",
    "type": "category",
    "state": "category",
    "party": "category",
}

In [4]:
# read csv file
df = pd.read_csv(
    "legislators-historical.csv",
    dtype=dtypes,
    usecols=list(dtypes) + ["birthday", "last_name"],
    parse_dates=["birthday"]
)

The dataset contains members’ first and last names, birth date, gender, type ("rep" for House of Representatives or "sen" for Senate), U.S. state, and political party. 

In [5]:
# view the first few rows of the dataset
df.head()

,last_name,first_name,birthday,gender,type,state,party
0,Bassett,Richard,1745-04-02,M,sen,DE,Anti-Administration
1,Bland,Theodorick,1742-03-21,M,rep,VA,NaN
2,Burke,Aedanus,1743-06-16,M,rep,SC,NaN
3,Carroll,Daniel,1730-07-22,M,rep,MD,NaN
4,Clymer,George,1739-03-16,M,rep,PA,NaN


In [6]:
# view the last few rows of the dataset
df.tail()

,last_name,first_name,birthday,gender,type,state,party
12037,Horn,Kendra,1976-06-09,F,rep,OK,Democrat
12038,Cunningham,Joe,1982-05-26,M,rep,SC,Democrat
12039,McAdams,Ben,1974-12-05,M,rep,UT,Democrat
12040,Riggleman,Denver,1970-03-01,M,rep,VA,Republican
12041,Hall,Kwanza,1971-05-01,M,rep,GA,Democrat


In [7]:
# The DataFrame uses categorical dtypes for space efficiency:
df.dtypes

# most columns have datatype category which reduces memory load on your machine

last_name             object
first_name          category
birthday      datetime64[ns]
gender              category
type                category
state               category
party               category
dtype: object

### Q: What is the count of Congressional members, on a state-by-state basis, over the entire history of the dataset?

Example SQL
```
SELECT state, count(name)
FROM df
GROUP BY state
ORDER BY state;
```

You call `.groupby()` and pass the name of the column you want to group on, which is "state". Then, you use `["last_name"]` to specify the columns on which you want to perform the actual aggregation.

In [7]:
n_by_state = df.groupby("state").count()
#n_by_state
n_by_state.head(10)

,last_name,first_name,birthday,gender,type,party
state,,,,,,
AK,16,16,16,16,16,14
AL,209,209,200,209,209,209
AR,117,117,114,117,117,115
AS,2,2,2,2,2,2
AZ,49,49,49,49,49,49
CA,366,366,363,366,366,366
CO,92,92,92,92,92,92
CT,240,240,238,240,240,227
DC,2,2,2,2,2,2


In [12]:
n_by_state = df.groupby("state")["last_name"].count()
#n_by_state
n_by_state.head(10)

state
AK     16
AL    209
AR    117
AS      2
AZ     49
CA    366
CO     92
CT    240
DC      2
DE     97
Name: last_name, dtype: int64

You can pass a lot more than just a single column name to .groupby() as the first argument. You can also specify any of the following:

- A list of multiple column names
- A dict or Pandas Series
- A NumPy array or Pandas Index, or an array-like iterable of these

### Example: Grouping jointly on 2 columns

Finds the count of Congressional members broken out by state and then by gender.

Example SQL

```
SELECT state, gender, count(name)
FROM df
GROUP BY state, gender
ORDER BY state, gender;
```

In [11]:
df.groupby(["state", "gender"])["last_name"].count()

state  gender
AK     F           0
       M          16
AL     F           4
       M         205
AR     F           5
                ... 
WI     M         198
WV     F           1
       M         119
WY     F           1
       M          39
Name: last_name, Length: 116, dtype: int64

In the Pandas version, the grouped-on columns are pushed into the MultiIndex of the resulting Series by default.


In [17]:
n_by_state_gender = df.groupby(["state", "gender"])["last_name"].count()
type(n_by_state_gender)

pandas.core.series.Series

In [18]:
n_by_state_gender.index[:5]

MultiIndex([('AK', 'F'),
            ('AK', 'M'),
            ('AL', 'F'),
            ('AL', 'M'),
            ('AR', 'F')],
           names=['state', 'gender'])

To more closely emulate the SQL result and push the grouped-on columns back into columns in the result, you an use `as_index=False`.

In [19]:
df.groupby(["state", "gender"], as_index=False)["last_name"].count()

,state,gender,last_name
0,AK,F,NaN
1,AK,M,16.0
2,AL,F,4.0
3,AL,M,205.0
4,AR,F,5.0
...,...,...,...
111,WI,M,198.0
112,WV,F,1.0
113,WV,M,119.0
114,WY,F,1.0


This produces a DataFrame with three columns and a RangeIndex, rather than a Series with a MultiIndex. In short, using `as_index=False` will make your result more closely mimic the default SQL output for a similar operation.

Note: In df.groupby(["state", "gender"])["last_name"].count(), you could also use .size() instead of .count(), since you know that there are no NaN last names. Using .count() excludes NaN values, while .size() includes everything, NaN or not.

In [20]:
df.groupby(["state", "gender"])["last_name"].size()

state  gender
AK     F           0
       M          16
AL     F           4
       M         205
AR     F           5
                ... 
WI     M         198
WV     F           1
       M         119
WY     F           1
       M          39
Name: last_name, Length: 116, dtype: int64

The SQL queries above explicitly use ORDER BY, whereas `.groupby()` does not. That’s because `.groupby()` does this by default through its parameter `sort`, which is `True` unless you tell it otherwise:

In [21]:
# Don't sort results by the sort keys
df.groupby("state", sort=False)["last_name"].count()

state
DE      97
VA     433
SC     251
MD     305
PA    1053
MA     427
NJ     359
GA     316
NY    1468
NC     356
CT     240
VT     115
KY     373
RI     107
NH     181
TN     301
OH     674
MS     155
OL       2
IN     343
LA     198
IL     488
MO     334
AL     209
AR     117
ME     175
FL     160
MI     296
IA     205
WI     198
TX     262
CA     366
OR      90
MN     161
NM      56
NE     127
WA      96
KS     143
UT      55
NV      56
CO      92
WV     120
DK       9
AZ      49
ID      59
MT      53
WY      40
DC       2
ND      44
SD      51
OK      93
HI      24
PR      19
AK      16
PI      13
VI       4
GU       4
AS       2
Name: last_name, dtype: int64

Next, you’ll dive into the object that .groupby() actually produces.

### How Pandas GroupBy Works

Before you get any further into the details, take a step back to look at `.groupby()` itself:

In [22]:
by_state = df.groupby("state")
print(by_state)

A `DataFrameGroupBy` object can be difficult understand because it’s lazy in nature. It doesn’t really do any operations to produce a useful result until you say so.

One term that’s frequently used alongside .groupby() is **split-apply-combine**.
This refers to a chain of three steps:

1. **Split** a table into groups
2. **Apply** some operations to each of those smaller tables
3. **Combine** the results

One useful way to inspect a Pandas GroupBy object and see the splitting in action is to iterate over it. This is implemented in `DataFrameGroupBy.__iter__()` and produces an `iterator` of (group, DataFrame) pairs for DataFrames.


In [25]:
for state, frame in by_state:
    print(f"First 2 entries for {state!r}")
    print("------------------------")
    print(frame.head(2), end="\n\n")

First 2 entries for 'AK'
------------------------
     last_name first_name   birthday gender type state        party
6617    Waskey      Frank 1875-04-20      M  rep    AK     Democrat
6645      Cale     Thomas 1848-09-17      M  rep    AK  Independent

First 2 entries for 'AL'
------------------------
    last_name first_name   birthday gender type state       party
911   Crowell       John 1780-09-18      M  rep    AL  Republican
990    Walker       John 1783-08-12      M  sen    AL  Republican

First 2 entries for 'AR'
------------------------
     last_name first_name   birthday gender type state party
1000     Bates      James 1788-08-25      M  rep    AR   NaN
1278    Conway      Henry 1793-03-18      M  rep    AR   NaN

First 2 entries for 'AS'
------------------------
          last_name first_name   birthday gender type state     party
10795         Sunia       Fofó 1937-03-13      M  rep    AS  Democrat
11752  Faleomavaega        Eni 1943-08-15      M  rep    AS  Democrat

F

If you’re working on a challenging aggregation problem, then iterating over the Pandas GroupBy object can be a great way to visualize the **split** part of **split-apply-combine**.

There are a few other methods and properties that let you look into the individual groups and their splits. The `.groups` attribute will give you a dictionary of `{group name: group label}` pairs. For example, `by_state` is a `dict` with states as keys. Here’s the value for the "PA" key:



In [26]:
by_state.groups["PA"]

Int64Index([    4,    19,    21,    27,    38,    57,    69,    76,    84,
               88,
            ...
            11838, 11862, 11871, 11873, 11883, 11887, 11926, 11938, 11952,
            11966],
           dtype='int64', length=1053)

Each value is a sequence of the index locations for the rows belonging to that particular group. In the output above, 4, 19, and 21 are the first indices in df at which the state equals “PA.”

You can also use `.get_group()` as a way to drill down to the sub-table from a single group:

In [27]:
by_state.get_group("PA")

,last_name,first_name,birthday,gender,type,state,party
4,Clymer,George,1739-03-16,M,rep,PA,NaN
19,Maclay,William,1737-07-20,M,sen,PA,Anti-Administration
21,Morris,Robert,1734-01-20,M,sen,PA,Pro-Administration
27,Wynkoop,Henry,1737-03-02,M,rep,PA,NaN
38,Jacobs,Israel,1726-06-09,M,rep,PA,NaN
...,...,...,...,...,...,...,...
11887,Brady,Robert,1945-04-07,M,rep,PA,Democrat
11926,Shuster,Bill,1961-01-10,M,rep,PA,Republican
11938,Rothfus,Keith,1962-04-25,M,rep,PA,Republican
11952,Costello,Ryan,1976-09-07,M,rep,PA,Republican


This is virtually equivalent to using `.loc[]`. You could get the same output with something like `df.loc[df["state"] == "PA"]`.

In [29]:
df.loc[df["state"] == "PA"]

,last_name,first_name,birthday,gender,type,state,party
4,Clymer,George,1739-03-16,M,rep,PA,NaN
19,Maclay,William,1737-07-20,M,sen,PA,Anti-Administration
21,Morris,Robert,1734-01-20,M,sen,PA,Pro-Administration
27,Wynkoop,Henry,1737-03-02,M,rep,PA,NaN
38,Jacobs,Israel,1726-06-09,M,rep,PA,NaN
...,...,...,...,...,...,...,...
11887,Brady,Robert,1945-04-07,M,rep,PA,Democrat
11926,Shuster,Bill,1961-01-10,M,rep,PA,Republican
11938,Rothfus,Keith,1962-04-25,M,rep,PA,Republican
11952,Costello,Ryan,1976-09-07,M,rep,PA,Republican


Next, what about the apply part? You can think of this step of the process as applying the same operation (or callable) to every “sub-table” that is produced by the splitting stage. 

From the Pandas GroupBy object `by_state`, you can grab the initial U.S. state and `DataFrame` with `next()`. When you iterate over a Pandas GroupBy object, you’ll get pairs that you can unpack into two variables:

In [31]:
state, frame = next(iter(by_state))  # First tuple from iterator
state

'AK'

In [32]:
frame.head(3)

,last_name,first_name,birthday,gender,type,state,party
6617,Waskey,Frank,1875-04-20,M,rep,AK,Democrat
6645,Cale,Thomas,1848-09-17,M,rep,AK,Independent
7440,Grigsby,George,1874-12-02,M,rep,AK,NaN


Now, think back to your original, full operation:

In [33]:
df.groupby("state")["last_name"].count()

state
AK      16
AL     209
AR     117
AS       2
AZ      49
CA     366
CO      92
CT     240
DC       2
DE      97
DK       9
FL     160
GA     316
GU       4
HI      24
IA     205
ID      59
IL     488
IN     343
KS     143
KY     373
LA     198
MA     427
MD     305
ME     175
MI     296
MN     161
MO     334
MS     155
MT      53
NC     356
ND      44
NE     127
NH     181
NJ     359
NM      56
NV      56
NY    1468
OH     674
OK      93
OL       2
OR      90
PA    1053
PI      13
PR      19
RI     107
SC     251
SD      51
TN     301
TX     262
UT      55
VA     433
VI       4
VT     115
WA      96
WI     198
WV     120
WY      40
Name: last_name, dtype: int64

The apply stage, when applied to your single, subsetted DataFrame, would look like this:

In [34]:
frame["last_name"].count()  # Count for state == 'AK'

16

You can see that the result, 16, matches the value for AK in the combined result.

The last step, **combine**, is the most self-explanatory. It simply takes the results of all of the applied operations on all of the sub-tables and combines them back together in an intuitive way.